In [1]:
import json, csv, os, ast
from os import listdir
from os.path import isfile, join
import pandas as pd
import numpy as np
import scipy as sp
from IPython.display import display, HTML
import xml.etree.ElementTree
import seaborn as sns
import pylab as plt
from matplotlib.backends.backend_pdf import PdfPages

%matplotlib inline

sns.set_style('whitegrid')

import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

In [2]:
df6 = pd.read_csv('../../Task/BigTask/Parsed-results_S6.csv')
df100 = pd.read_csv('../../Task/BigTask/Parsed-results_S100.csv')

In [5]:
# Funzione per Estrapolare il Dominio
def find_domain(url):
    st = url.split("/")
    return st[2]

In [6]:
# S100
df100['url'][80] = "http://www.fakeURL.com"
df100['url'][373] = "http://www.fakeURL.com"
# Prendo i domini
df100['url'] = df100.apply(lambda row: find_domain(row.url), axis = 1)
df100.head(2)

c:\users\barbe\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
c:\users\barbe\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


IndexError: ('list index out of range', 'occurred at index 0')

In [7]:
df6['url'] = df6.apply(lambda row: find_domain(row.url), axis = 1)
df6.head(2)

,unit_id,doc_id,rel,worker_id,gender,age,school,income,party,party_ln,ideo,elections,teaparty,position_in_task,S6_rel,justification,url,off_page_count,ip_address,time
0,unit_0,REP_LIE_doc6,LIE,AFBS8D6KBKKKY,2,3,4,3,2,2,4,1,3,1,1,"After implementation of the law, 98% of Massac...",en.wikipedia.org,0,65.35.157.98,105.615
1,unit_0,DEM_FALSE_doc1,FALSE,AFBS8D6KBKKKY,2,3,4,3,2,2,4,1,3,2,1,She has literally cited him in the past.,twitchy.com,3,65.35.157.98,133.090


In [8]:
my_df100 = df100[['unit_id', 'doc_id', 'url']]
my_df100.columns = ['unit_id', 'doc_id', 'url_S100']
display(my_df100.head(8))
my_df6 = df6[['unit_id', 'doc_id', 'url']]
my_df6.columns = ['unit_id', 'doc_id', 'url_S6']
merged = pd.merge(my_df100, my_df6, how = 'inner', left_on=['unit_id','doc_id'], right_on = ['unit_id','doc_id'])

my_df100['count_S100'] = 0
my_df6['count_S6'] = 0
my_df100 = my_df100.groupby(['unit_id', 'url_S100']).count().reset_index()
my_df6 = my_df6.groupby(['unit_id', 'url_S6']).count().reset_index()
grouped_merged = pd.merge(my_df100, my_df6, how = 'inner', on = ['doc_id'])

display(merged.head(20))
display(grouped_merged.head())

,unit_id,doc_id,url_S100
0,unit_0,REP_LIE_doc6,en.wikipedia.org
1,unit_0,DEM_FALSE_doc1,www.facebook.com
2,unit_0,LOW,www.anb.org
3,unit_0,DEM_HALFTRUE_doc3,www.politifact.com
4,unit_0,DEM_TRUE_doc2,www.vanityfair.com
5,unit_0,HIGH,www.youtube.com
6,unit_0,REP_MOSTLYTRUE_doc5,www.politifact.com
7,unit_0,REP_BARELYTRUE_doc4,www.texasstandard.org


c:\users\barbe\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
c:\users\barbe\anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


,unit_id,doc_id,url_S100,url_S6
0,unit_0,REP_LIE_doc6,en.wikipedia.org,en.wikipedia.org
1,unit_0,DEM_FALSE_doc1,www.facebook.com,twitchy.com
2,unit_0,LOW,www.anb.org,en.wikipedia.org
3,unit_0,DEM_HALFTRUE_doc3,www.politifact.com,www.politifact.com
4,unit_0,DEM_TRUE_doc2,www.vanityfair.com,www.vanityfair.com
5,unit_0,HIGH,www.youtube.com,www.huffpost.com
6,unit_0,REP_MOSTLYTRUE_doc5,www.politifact.com,www.politifact.com
7,unit_0,REP_BARELYTRUE_doc4,www.texasstandard.org,www.politifact.com
8,unit_1,REP_TRUE_doc8,www.politifact.com,en.wikipedia.org
9,unit_1,LOW,en.wikipedia.org,www.battlefields.org


,unit_id_x,url_S100,doc_id,count_S100,unit_id_y,url_S6,count_S6
0,unit_0,en.wikipedia.org,1,1,unit_0,twitchy.com,1
1,unit_0,en.wikipedia.org,1,1,unit_0,www.huffpost.com,1
2,unit_0,en.wikipedia.org,1,1,unit_0,www.vanityfair.com,1
3,unit_0,en.wikipedia.org,1,1,unit_1,www.battlefields.org,1
4,unit_0,en.wikipedia.org,1,1,unit_1,www.johnmccain.com,1


In [9]:
def check_same(id1, id2):
    if(id1 == id2):
        return True
    else:
        return False
    
merged['same'] = merged.apply(lambda row: check_same(row.url_S100, row.url_S6), axis = 1)
display(merged.head())
gold_dict = ['HIGH', 'LOW']
merged = merged[merged["doc_id"].isin(gold_dict) == False]

same_count = merged.groupby(['same']).count().reset_index()
display(same_count)

,unit_id,doc_id,url_S100,url_S6,same
0,unit_0,REP_LIE_doc6,en.wikipedia.org,en.wikipedia.org,True
1,unit_0,DEM_FALSE_doc1,www.facebook.com,twitchy.com,False
2,unit_0,LOW,www.anb.org,en.wikipedia.org,False
3,unit_0,DEM_HALFTRUE_doc3,www.politifact.com,www.politifact.com,True
4,unit_0,DEM_TRUE_doc2,www.vanityfair.com,www.vanityfair.com,True


,same,unit_id,doc_id,url_S100,url_S6
0,False,518,518,518,518
1,True,680,680,680,680


In [10]:
gold_dict = ['HIGH', 'LOW']
merged = merged[merged["doc_id"].isin(gold_dict) == False]

merged = merged[['doc_id', 'url_S100', 'url_S6']]
#merged = merged.set_index('doc_id')
merged

,doc_id,url_S100,url_S6
0,REP_LIE_doc6,en.wikipedia.org,en.wikipedia.org
1,DEM_FALSE_doc1,www.facebook.com,twitchy.com
3,DEM_HALFTRUE_doc3,www.politifact.com,www.politifact.com
4,DEM_TRUE_doc2,www.vanityfair.com,www.vanityfair.com
6,REP_MOSTLYTRUE_doc5,www.politifact.com,www.politifact.com
7,REP_BARELYTRUE_doc4,www.texasstandard.org,www.politifact.com
8,REP_TRUE_doc8,www.politifact.com,en.wikipedia.org
10,REP_FALSE_doc2,www.politifact.com,en.wikipedia.org
11,DEM_BARELYTRUE_doc9,www.politifact.com,en.wikipedia.org
12,REP_HALFTRUE_doc10,www.politifact.com,www.johnmccain.com
